### Notebook summary
In this notebook we have trained a model with only 6 important features selected by ```sklearn``` library's ```SelectKBest``` for AAPL ticker using SPO framework.
Below points were observed:
- Even with reduced features loss is in higher magnitude
- Variance in loss is also high

Next steps:
- Reduce features based on domain knowledge

### Import libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import itertools
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import random
import gurobipy as gp
from gurobipy import GRB
import tensorflow as tf
from tensorflow.keras import initializers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
import yaml
from pathlib import Path

### Import modules

In [ ]:
import sys
sys.path.append("../src")

import data_exploration as de
import model_training as mt

### Load necessary directories

In [ ]:
current_dir = Path(os.getcwd())
root_dir = current_dir
while 'Portfolio Optimization using SPO' in root_dir.parts:
    root_dir = root_dir.parent
    if root_dir == Path(root_dir.root):
        print("Root directory not found.")
        break

In [ ]:
config_path = root_dir / "Portfolio Optimization using SPO" / "config" / "config.yml"
complete_data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "dat_518_companies.csv"
data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "AAPL_df.csv"
cost_mat_path = root_dir / "Portfolio Optimization using SPO" / "data" / "cost_mat.csv"
sigma_path = root_dir / "Portfolio Optimization using SPO" / "data" / "sigma_df.csv"

In [ ]:
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

### Import data

In [ ]:
# import data
df_AAPL_train_test = pd.read_csv(data_path)
df_final_returns = pd.read_csv(cost_mat_path)
sigma_df = pd.read_csv(sigma_path)

In [ ]:
gamma = config["gamma"]
sigma = sigma_df.values

### Calculate and plot feature importance score

In [ ]:
all_features = df_AAPL_train_test.iloc[:,0:n_feats]
target = df_AAPL_train_test.iloc[:,-1]
all_features_fs, fs = mt.select_features(all_features, target, k='all')
importance = list(fs.scores_)
col_names = list(all_features.columns)
fig = px.bar(x=col_names, y=importance).update_layout(title="Importance of each feature", xaxis_title="Features", yaxis_title="Mutual Importance Score")
fig.show()

### Select top 6 features and create a dataframe

In [ ]:
six_features_fs, fs_6 = mt.select_features(all_features, target, k=6)
mask = fs_6.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool_val, feature in zip(mask, col_names):
    if bool_val:
        new_features.append(feature)
        
df_train_six_imp = pd.DataFrame(six_features_fs, columns=new_features)
df_train_six_Y = pd.concat([df_train_six_imp, target], axis=1)

### Split data into train and test

In [ ]:
# training dataframe
df_AAPL_fi_train, df_AAPL_fi_test = train_test_split(df_train_six_Y, test_size=0.2, random_state=42, shuffle=False)

# cost vector
df_final_returns_train, df_final_returns_test = train_test_split(df_final_returns, test_size=0.2, random_state=42, 
                                                                 shuffle=False)

### Initialize the model

In [ ]:
fi_n_rows, fi_n_cols = df_AAPL_fi_train.shape
fi_n_feats = fi_n_cols-1

# Instantiate the model
model_fi_data = mt.get_model(n_feats = fi_n_feats)
model_fi_data.summary()

### Train the model
We will train the model with random hyper-parameters to test if everything is working fine.

In [ ]:
%%time
trained_fi_model, epoch_fi_loss_list = mt.SGD_regressor(df_AAPL_fi_train, model_fi_data, df_final_returns_train, sigma, gamma, learning_rate= 0.0001, decay_rate=1.02, n_epochs=200, batch_size = 256)

### Plot loss progression with every epoch

In [ ]:
fig_fi = px.line(epoch_fi_loss_list).update_layout(title="Training Loss progression", xaxis_title="epochs", yaxis_title="SPO+ loss")
fig_fi.show()

### Testing the model on test data

In [ ]:
y_pred_fi = trained_fi_model(df_AAPL_fi_test.iloc[:,0:fi_n_feats].values)
fi_spo_test_loss = mt.get_SPO_plus_testing_loss(df_AAPL_fi_train, df_final_returns_test, y_pred_fi, sigma=sigma, gamma=gamma)

print(f'The SPO+ loss on testing data is {fi_spo_test_loss}')

After observing the loss at every epoch and also on testing data, the loss has very high magnitue and high variability as well so we will do feature selection based on domain knowledge from the research by [Zhong and Hitchcock (2021)](https://github.com/Shanlearning/SP-500-Stock-Prediction/tree/master) to see if the loss can be reduced.